# Medical Report Summarization

This notebook demonstrates how to use models from Hugging Face for medical report summarization, how to build a user interface using Gradio, and how to incorporate text-to-speech generation. It is important to connect to T4 GPU runtime in Google Colab while running this notebook.

In [ ]:
# Install required dependencies silently
%pip install soundfile gradio transformers --quiet

## Medical Report Summarizer with Gradio UI

This code cell builds a complete Gradio application for medical report summarization and text-to-speech conversion.

**Objective:** To create a user-friendly web interface that allows users to input a medical report, get a summary, and then generate audio for the summary.

**Key Aspects and Steps:**

1.  **Import Libraries:** Imports necessary libraries including `gradio`, `transformers`, `torch`, `os`, and `time`.
2.  **Configuration:** Sets the Hugging Face model name for summarization and securely retrieves the Hugging Face token using Colab's userdata secrets.
3.  **Summarization Functionality:** Defines the `summarize_text` function using a Hugging Face pipeline to generate a summary from the input text.
4.  **Export Functionality:** Defines the `export_summary` function to save the generated summary to a text file, making it available for download.
5.  **Text-to-Speech Functionality:** Defines the `generate_voice` function, which utilizes the `suno/bark` model to convert the summary text into speech audio. It handles moving the model and inputs to the GPU if available for faster processing.
6.  **UI State Management:** Includes the `update_component_states` function to dynamically control the visibility and interactivity of Gradio components (like the export and audio buttons) based on whether a summary has been generated.
7.  **Gradio Interface:** Builds the Gradio interface using `gr.Blocks` for flexible layout control. It includes input textboxes, buttons for summarize and clear, output areas for the summary and audio, and a download link for the exported file.
8.  **Event Handlers:** Defines the actions that occur when buttons are clicked, linking them to the respective Python functions (`summarize_text`, `export_summary`, `generate_voice`, `update_component_states`).
9.  **Launch App:** Launches the Gradio web application, making it accessible via a public URL in the Colab environment.

This setup demonstrates a complete workflow from text input to summarized output, audio generation, and file export, all managed within a interactive Gradio interface. The component visibility and interactivity are controlled to provide a guided user experience.

In [ ]:
# Import necessary libraries
import gradio as gr
from transformers import pipeline, AutoProcessor, AutoModel
import os
import time
import torch # Import torch

# Securely retrieve Hugging Face token from Colab userdata secrets
# You can set this in the Colab secrets manager under the "🔑" icon
from google.colab import userdata

# --- Configuration ---
# Step 1: Specify the Hugging Face model for summarization
# Example: "facebook/bart-large-cnn" or "t5-small"
MODEL_NAME = "Falconsai/medical_summarization"

# Retrieve Hugging Face token
# Not recommended to paste tokens directly in public notebooks
HF_TOKEN = userdata.get('HF_AIC2_TOKEN')

# --- Summarization Functionality ---
# Step 2: Load the summarization pipeline
summarizer = pipeline("summarization", model=MODEL_NAME)

# A sample medical report for default input
sample_report = """
Reason for Evaluation:
The patient was referred for a medical evaluation following a motor vehicle accident that occurred on September 15, 2025, to assess ongoing musculoskeletal complaints and determine functional limitations, if any.
History of Present Illness:
Mr. Doe reports persistent pain in the lower back and right shoulder since the incident. Pain intensity is rated 6/10 on average, aggravated by bending, prolonged sitting, or lifting. He denies any numbness or weakness in the extremities.
Past Medical History:
- no prior history of spinal or shoulder injury
- No chronic medical conditions reported
- Non-smoker, occasional alcohol use
Physical Examination:
- General: Patient alert and oriented; normal gait observed.
- Spine: Mild tenderness over the lumbosacral area; range of motion mildly restricted in flexion and extension.
- Right Shoulder: Mild tenderness over the acromioclavicular joint; range of motion limited to 150° abduction with pain.
- Neurological: Reflexes intact; no motor or sensory deficits noted.
Imaging & Diagnostic Tests:
- X-ray Lumbar Spine: No fracture; mild degenerative changes noted.
- MRI Shoulder (Right): Mild supraspinatus tendonitis without tear.
"""

# Step 3: Define the function to summarize text
def summarize_text(text):
    """
    Summarizes the input text using the configured Hugging Face model.

    Args:
        text (str): The input text to be summarized.

    Returns:
        str: The summarized text or an error message if input is empty.
    """
    if not text.strip():
        return "⚠️ Please provide a medical document to summarize."
    # Perform summarization
    summary = summarizer(text, max_length=300, min_length=75, do_sample=False)
    return summary[0]["summary_text"]

# --- Export Functionality ---
# Step 4: Define the function to export the summary to a text file
def export_summary(summary):
    """
    Exports the given summary text to a file named 'medical_summary.txt'.

    Args:
        summary (str): The text summary to export.

    Returns:
        str: The filename of the exported file.
    """
    output_filename = "medical_summary.txt"
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(summary)
    # Return the filename which Gradio can use to create a download link
    return output_filename

# --- Text-to-Speech Functionality ---
# Additional steps to convert summary text to Speech Audio
def generate_voice(summary):
  """
  Generates speech audio from the input summary text using the suno/bark model.

  Args:
      summary (str): The text summary to convert to speech.

  Returns:
      tuple: A tuple containing the sampling rate and the audio data as a numpy array,
             formatted for Gradio's Audio component.
  """
  print("--- Entering generate_voice function ---")
  start_time = time.time()

  # Load the Bark model and processor
  processor = AutoProcessor.from_pretrained("suno/bark")
  model = AutoModel.from_pretrained("suno/bark")
  print(f"Model and processor loaded in {time.time() - start_time:.2f} seconds.")

  # Process the input text
  inputs = processor(
      text=[summary],
      return_tensors="pt",
  )
  print(f"Inputs processed in {time.time() - start_time:.2f} seconds.")

  # Move model and inputs to GPU if available for faster processing
  if torch.cuda.is_available():
      model = model.to("cuda")
      inputs = inputs.to("cuda")
      print("Moved model and inputs to GPU.")

  # Generate speech audio
  speech_values = model.generate(**inputs, do_sample=True)
  print(f"Speech generated in {time.time() - start_time:.2f} seconds.")

  # Get the sampling rate from the model config
  sampling_rate = model.generation_config.sample_rate
  print(f"Sampling rate obtained in {time.time() - start_time:.2f} seconds.")

  print("Audio generation finished.")
  # Gradio's Audio component expects a tuple of (sampling_rate, audio_data)
  return (sampling_rate, speech_values[0].cpu().numpy().squeeze())

# --- UI State Management ---
# Function to update component states based on summary text presence
def update_component_states(summary):
    """
    Updates the visibility and interactivity of UI components based on whether a summary exists.

    Args:
        summary (str): The current summary text.

    Returns:
        tuple: A tuple of updated Gradio components.
    """
    # Enable/disable components based on if summary output is not empty
    is_enabled = bool(summary.strip())
    return gr.Textbox(label="Summary Output", lines=10, value=summary, visible=is_enabled), \
           gr.Button("💾 Export Summary", interactive=is_enabled), \
           gr.Audio(label="Audio Output", visible=is_enabled), \
           gr.Button("🎤 Generate Audio", visible=is_enabled)


# --- Gradio Interface ---
# Step 5: Build the Gradio interface using Blocks for more control over layout
with gr.Blocks(theme=gr.themes.Citrus()) as demo:
    gr.Markdown("## 📝 Medical Report Summarizer ")
    gr.Markdown("Enter your medical report below and click **Summarize**.")

    # Input and control row
    with gr.Row():
        text_input = gr.Textbox(
            label="Input Text",
            placeholder="Paste a medical report here...",
            lines=10,
            value=sample_report
        )
    with gr.Row():
        summarize_button = gr.Button("🔍 Summarize")
        clear_button = gr.Button("🧹 Clear")

    # Output row (Summary and Audio)
    with gr.Row():
      output_box = gr.Textbox(label="Summary Output", lines=20, visible=False)
      audio_output = gr.Audio(label="Audio Output", visible=False)

    # Action buttons row (Export and Generate Audio)
    with gr.Row():
      export_button = gr.Button("💾 Export Summary", interactive=False)
      generate_audio_button = gr.Button("🎤 Generate Audio", visible=False) # Button for audio generation

    # File component for download
    output_file = gr.File(label="Exported Summary", visible=False)

    # --- Define Actions (Event Handlers) ---
    # Action for the Summarize button
    summarize_button.click(
        fn=summarize_text,
        inputs=text_input,
        outputs=output_box
    )

    # Action for the Clear button
    clear_button.click(
        fn=lambda: "", # Use a lambda to clear the textboxes
        inputs=None,
        outputs=[text_input, output_box]
    )

    # Action for the Export button
    export_button.click(
        fn=export_summary,
        inputs=output_box,
        outputs=[output_file] # Output the file component
    )

    # Action for the Generate Audio button
    print("--- Setting up generate_audio_button click event ---") # Debug print
    generate_audio_button.click(
        fn=generate_voice,
        inputs=output_box,
        outputs=audio_output # Output the audio component
    )

    # Update component states when the summary output changes
    # This ensures buttons are enabled only when there is a summary
    output_box.change(
        fn=update_component_states,
        inputs=output_box,
        outputs=[output_box, export_button, audio_output, generate_audio_button]
    )

    # Optional: Markdown for theme switching hint
    gr.Markdown("---")
    gr.Markdown("🌗 Switch between light and dark themes in the top-right corner.")
    gr.Markdown("🎤 While generating audio summary, please wait for 4-6 mins for audio generation.")

# --- Launch the Gradio App ---
# Step 6: Launch the Gradio demo
if __name__ == "__main__":
    # launch() will automatically set share=True in Colab
    demo.launch()